In [ ]:
import getpass
import os
password = getpass.getpass()
command = "sudo -S apt update"
os.system('echo %s | %s' % (password, command))

command = "sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null"
os.system('echo %s | %s' % (password, command))
!
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
%pip install -q findspark
%pip install pyspark
%pip install py4j
%pip install requests


In [11]:

import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Projekt grupowy") \
       .getOrCreate()

spark

your 131072x1 screen size is bogus. expect trouble
24/12/13 19:40:48 WARN Utils: Your hostname, Rogal resolves to a loopback address: 127.0.1.1; using 172.20.196.165 instead (on interface eth0)
24/12/13 19:40:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 19:40:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
search_phrase = '5g'

api_call = f'https://dblp.org/search/publ/api?q={search_phrase}&format=json'

In [13]:
spark.sql("""CREATE TABLE IF NOT EXISTS authors (pid string, author string) USING PARQUET""")

DataFrame[]

In [14]:
spark.sql("""CREATE TABLE IF NOT EXISTS publications (title string, authors_pid array<string>, year string) USING PARQUET""")

DataFrame[]

In [15]:
import requests
import pyspark.sql
authors = []
publications = []
data = requests.get(api_call).json()
data

{'result': {'query': '5g',
  'status': {'@code': '200', 'text': 'OK'},
  'time': {'@unit': 'msecs', 'text': '4.25'},
  'completions': {'@total': '1',
   '@computed': '1',
   '@sent': '1',
   'c': {'@sc': '19256',
    '@dc': '18679',
    '@oc': '19256',
    '@id': '56877516',
    'text': '5g'}},
  'hits': {'@total': '18679',
   '@computed': '100',
   '@sent': '30',
   '@first': '0',
   'hit': [{'@score': '4',
     '@id': '985736',
     'info': {'authors': {'author': {'@pid': '214/4100',
        'text': 'Thomas Deinlein'}},
      'title': 'Simulation und Modellierung von 5G Vehicle-to-Infrastructure (V2I) / Vehicle-to-Network (V2N) Anwendungsfällen unter Berücksichtigung des 5G QoS Modells (Simulation and modeling of 5G Vehicle-to-Infrastructure (V2I) / Vehicle-to-Network (V2N) use cases under consideration of the 5G QoS model)',
      'year': '2022',
      'type': 'Books and Theses',
      'access': 'closed',
      'key': 'phd/basesearch/Deinlein22',
      'ee': 'https://opus4.kobv.de/o

In [16]:
len(data['result']['hits']['hit'])

30

In [17]:
publications = []
authors = []

In [18]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType
authors_schema = StructType([
    StructField('pid', StringType(), True),
    StructField('author', StringType(), True),
])

publications_schema = StructType([
    StructField('title', StringType(), True),
    StructField('authors_pid', ArrayType(StringType()), True),
    StructField('year', StringType(), True)
])


for x in range(len(data['result']['hits']['hit'])):
  hit = data['result']['hits']['hit'][x]

  try:
    if hit['info'].get('authors') is None:
      continue
    if isinstance(hit['info']['authors']['author'], list):
      pid = []
      for y in range(len(hit['info']['authors']['author'])):
        authors.append([hit['info']['authors']['author'][y]['@pid'], hit['info']['authors']['author'][y]['text']])
        pid.append(hit['info']['authors']['author'][y]['@pid'])
      publications.append([hit['info']['title'], pid, hit['info']['year']])
    elif isinstance(hit['info']['authors']['author'], dict):
      authors.append([hit['info']['authors']['author']['@pid'], hit['info']['authors']['author']['text']])
      publications.append([hit['info']['title'], [hit['info']['authors']['author']['@pid']], hit['info']['year']])
  except Exception as e:
    print('Error: ', e)


authors_df = spark.createDataFrame(data=authors, schema=authors_schema)
publications_df = spark.createDataFrame(data=publications, schema=publications_schema)

In [19]:
authors_df.write.mode('append').saveAsTable("authors")


24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 75.08% for 9 writers
24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 67.58% for 10 writers
24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 61.43% for 11 writers
24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 56.31% for 12 writers
24/12/13 19:41:19 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,

In [20]:
publications_df.write.mode('append').saveAsTable("publications")

24/12/13 19:41:24 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [21]:
from pyspark.sql.functions import array_contains
authors_df.join(publications_df, array_contains(publications_df.authors_pid, authors_df.pid), 'inner').select("pid", "author", "title", "year").show(1000)

+-----------------+--------------------+--------------------+----+
|              pid|              author|               title|year|
+-----------------+--------------------+--------------------+----+
|         214/4100|     Thomas Deinlein|Simulation und Mo...|2022|
|         282/9003|       Anouar Rkhami|From servies plac...|2021|
|         185/5677|Konstantinos Kousias|Proceedings of th...|2024|
|         54/476-1|      Feng Qian 0001|Proceedings of th...|2024|
|         204/4378|      Daniel Silhavy|3GPP Rel-17 5G Me...|2023|
|         355/2053|        David Waring|3GPP Rel-17 5G Me...|2023|
|         02/11004|          Dev Audsin|3GPP Rel-17 5G Me...|2023|
|         355/3143|    Richard Bradbury|3GPP Rel-17 5G Me...|2023|
|         325/4666|         Johann Mika|3GPP Rel-17 5G Me...|2023|
|         325/4623|   Klaus Kuehnhammer|3GPP Rel-17 5G Me...|2023|
|         196/5823|         Kurt Krauss|3GPP Rel-17 5G Me...|2023|
|         80/10608|  Jordi Joan Gimenez|3GPP Rel-17 5G Me...|2

In [22]:
spark.sql("SELECT a.pid, a.author, p.title, p.year FROM authors a INNER JOIN publications p ON array_contains(p.authors_pid, pid)").show()

+------------+--------------------+--------------------+----+
|         pid|              author|               title|year|
+------------+--------------------+--------------------+----+
|     66/8170|Konstantinos Kats...|Enhancing Tourist...|2019|
|    156/0162|   Dimitrios Gkounis|Enhancing Tourist...|2019|
|      26/803|      Dritan Kaleshi|Enhancing Tourist...|2019|
|    235/7968|          Ben Thomas|Enhancing Tourist...|2019|
|      62/255|         John Harris|Enhancing Tourist...|2019|
|    254/5745|        Hamid Falaki|Enhancing Tourist...|2019|
|     23/5432|  Dimitra Simeonidou|Enhancing Tourist...|2019|
|j/RikuJantti|         Riku Jäntti|PriMO-5G: making ...|2019|
|      39/429|        Minseok Choi|PriMO-5G: making ...|2019|
|    247/7521|       Joohyung Jeon|PriMO-5G: making ...|2019|
|   93/5867-1|     Dohyun Kim 0001|PriMO-5G: making ...|2019|
|     69/4977|       Joongheon Kim|PriMO-5G: making ...|2019|
|     24/5197|  José Costa-Requena|PriMO-5G: making ...|2019|
|    254